# Keras Implementation of ArcFace, CosFace, AdaCos

In this notebook you may find the implementation of layers that you may need for this competition.

## Why Softmax and Triplet Loss is bad?
There are two main lines research to train CNN for face recognition, one that train a multi-class classifier using softmax classifier and the other that learn the embeddings such as the triplet loss. However both have their drawbacks:
 - For the softmax loss, the more you add the different identities for recognition, the more number of parameters will increase.
 - And for the triplet loss there is a combinatorial explosion in the number of face triplets for large scale dataset leading to large number of iterations.

Below is the visualization of Softmax trained on MNIST. As you can see, the points are quite scattered even for such a simple dataset. We will compare this to other losses in next sections.
![](https://github.com/4uiiurz1/keras-arcface/raw/master/figures/mnist_vgg8_3d.png)

In [ ]:
import math
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras.initializers import Constant
from tensorflow.python.keras.utils import tf_utils


def _resolve_training(layer, training):
    if training is None:
        training = K.learning_phase()
    if isinstance(training, int):
        training = bool(training)
    if not layer.trainable:
        # When the layer is not trainable, override the value
        training = False
    return tf_utils.constant_value(training)

# ArcFace

Paper source: https://arxiv.org/pdf/1801.07698.pdf

An additive angular margin loss is proposed in arcface to further improve the descriminative power of the face recognition model and stabilize the training process. The arc-cosine function is used to calculate angle between the current feature and target weight. ArcFace directly optimizes the geodesic distance margin by virtue of exact correspondence between angle and arc in the normalized hypersphere.

$$L_A = -\frac{1}{N} \sum_{i=1}^N \log \frac{e^{s(\cos (\theta_{y_i} + m))}}{e^{s(\cos (\theta_{y_i} + m))} + \sum_{j=1, j\ne y_i}^N e^{s \cos \theta_j}}$$

Below is the visualization for a network trained on MNIST with ArcFace.
![](https://github.com/4uiiurz1/keras-arcface/raw/master/figures/mnist_vgg8_arcface_3d.png)

In [ ]:
class ArcFace(Layer):
    """
    Implementation of ArcFace layer. Reference: https://arxiv.org/abs/1801.07698
    
    Arguments:
      num_classes: number of classes to classify
      s: scale factor
      m: margin
      regularizer: weights regularizer
    """
    def __init__(self,
                 num_classes,
                 s=30.0,
                 m=0.5,
                 regularizer=None,
                 name='arcface',
                 **kwargs):
        
        super().__init__(name=name, **kwargs)
        self._n_classes = num_classes
        self._s = float(s)
        self._m = float(m)
        self._regularizer = regularizer

    def build(self, input_shape):
        embedding_shape, label_shape = input_shape
        self._w = self.add_weight(shape=(embedding_shape[-1], self._n_classes),
                                  initializer='glorot_uniform',
                                  trainable=True,
                                  regularizer=self._regularizer,
                                  name='cosine_weights')

    def call(self, inputs, training=None):
        """
        During training, requires 2 inputs: embedding (after backbone+pool+dense),
        and ground truth labels. The labels should be sparse (and use
        sparse_categorical_crossentropy as loss).
        """
        embedding, label = inputs

        # Squeezing is necessary for Keras. It expands the dimension to (n, 1)
        label = tf.reshape(label, [-1], name='label_shape_correction')

        # Normalize features and weights and compute dot product
        x = tf.nn.l2_normalize(embedding, axis=1, name='normalize_prelogits')
        w = tf.nn.l2_normalize(self._w, axis=0, name='normalize_weights')
        cosine_sim = tf.matmul(x, w, name='cosine_similarity')

        training = resolve_training_flag(self, training)
        if not training:
            # We don't have labels if we're not in training mode
            return self._s * cosine_sim
        else:
            one_hot_labels = tf.one_hot(label,
                                        depth=self._n_classes,
                                        name='one_hot_labels')
            theta = tf.math.acos(K.clip(
                    cosine_sim, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
            selected_labels = tf.where(tf.greater(theta, math.pi - self._m),
                                       tf.zeros_like(one_hot_labels),
                                       one_hot_labels,
                                       name='selected_labels')
            final_theta = tf.where(tf.cast(selected_labels, dtype=tf.bool),
                                   theta + self._m,
                                   theta,
                                   name='final_theta')
            output = tf.math.cos(final_theta, name='cosine_sim_with_margin')
            return self._s * output

# CosFace: Large Margin Cosine Loss

Paper source: https://arxiv.org/abs/1801.09414

Large Margin Cosine Loss (LMCL) which is referred as CosFace, reformulates the traditional softmax loss as a cosine loss by L2 normalizing both features and weight vectors to remove radial variations, based on this a cosine margin term is introduced to further maximize the decision margin in the angular space. As a result of which we get minimum intra-class margin and maximum inter-class margin for accurate face verification.

The formula for CosFace is pretty similar to ArcFace:

$$L_A = -\frac{1}{N} \sum_{i=1}^N \log \frac{e^{s(\cos (\theta_{y_i}) - m)}}{e^{s(\cos (\theta_{y_i}) - m)} + \sum_{j=1, j\ne y_i}^N e^{s \cos \theta_j}}$$

Below is the visualization for a network trained on MNIST with CosFace.

![](https://github.com/4uiiurz1/keras-arcface/raw/master/figures/mnist_vgg8_cosface_3d.png)

In [ ]:
class CosFace(Layer):
    """
    Implementation of CosFace layer. Reference: https://arxiv.org/abs/1801.09414
    
    Arguments:
      num_classes: number of classes to classify
      s: scale factor
      m: margin
      regularizer: weights regularizer
    """
    def __init__(self,
                 num_classes,
                 s=30.0,
                 m=0.35,
                 regularizer=None,
                 name='cosface',
                 **kwargs):

        super().__init__(name=name, **kwargs)
        self._n_classes = num_classes
        self._s = float(s)
        self._m = float(m)
        self._regularizer = regularizer

    def build(self, input_shape):
        embedding_shape, label_shape = input_shape
        self._w = self.add_weight(shape=(embedding_shape[-1], self._n_classes),
                                  initializer='glorot_uniform',
                                  trainable=True,
                                  regularizer=self._regularizer)

    def call(self, inputs, training=None):
        """
        During training, requires 2 inputs: embedding (after backbone+pool+dense),
        and ground truth labels. The labels should be sparse (and use
        sparse_categorical_crossentropy as loss).
        """
        embedding, label = inputs

        # Squeezing is necessary for Keras. It expands the dimension to (n, 1)
        label = tf.reshape(label, [-1], name='label_shape_correction')

        # Normalize features and weights and compute dot product
        x = tf.nn.l2_normalize(embedding, axis=1, name='normalize_prelogits')
        w = tf.nn.l2_normalize(self._w, axis=0, name='normalize_weights')
        cosine_sim = tf.matmul(x, w, name='cosine_similarity')

        training = _resolve_training(self, training)
        if not training:
            # We don't have labels if we're not in training mode
            return self._s * cosine_sim
        else:
            one_hot_labels = tf.one_hot(label,
                                        depth=self._n_classes,
                                        name='one_hot_labels')
            final_theta = tf.where(tf.cast(one_hot_labels, dtype=tf.bool),
                                   cosine_sim - self._m,
                                   cosine_sim,
                                   name='cosine_sim_with_margin')
            return self._s * output

# AdaCos: Adaptively Scaling Cosine Logits

Paper source: https://arxiv.org/abs/1905.00292

In [ ]:
class AdaCos(Layer):
    """
    Implementation of AdaCos layer. Reference: https://arxiv.org/abs/1905.00292
    
    Arguments:
      num_classes: number of classes to classify
      is_dynamic: if False, use Fixed AdaCos. Else, use Dynamic Adacos.
      regularizer: weights regularizer
    """
    def __init__(self,
                 num_classes,
                 is_dynamic=True,
                 regularizer=None,
                 **kwargs):

        super().__init__(**kwargs)
        self._n_classes = num_classes
        self._init_s = math.sqrt(2) * math.log(num_classes - 1)
        self._is_dynamic = is_dynamic
        self._regularizer = regularizer

    def build(self, input_shape):
        embedding_shape, label_shape = input_shape
        self._w = self.add_weight(shape=(embedding_shape[-1], self._n_classes),
                                  initializer='glorot_uniform',
                                  trainable=True,
                                  regularizer=self._regularizer)
        if self._is_dynamic:
            self._s = self.add_weight(shape=(),
                                      initializer=Constant(self._init_s),
                                      trainable=False,
                                      aggregation=tf.VariableAggregation.MEAN)

    def call(self, inputs, training=None):
        embedding, label = inputs

        # Squeezing is necessary for Keras. It expands the dimension to (n, 1)
        label = tf.reshape(label, [-1])

        # Normalize features and weights and compute dot product
        x = tf.nn.l2_normalize(embedding, axis=1)
        w = tf.nn.l2_normalize(self._w, axis=0)
        logits = tf.matmul(x, w)

        # Fixed AdaCos
        is_dynamic = tf_utils.constant_value(self._is_dynamic)
        if not is_dynamic:
            # _s is not created since we are not in dynamic mode
            output = tf.multiply(self._init_s, logits)
            return output

        training = _resolve_training(self, training)
        if not training:
            # We don't have labels to update _s if we're not in training mode
            return self._s * logits
        else:
            theta = tf.math.acos(
                    K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
            one_hot = tf.one_hot(label, depth=self._n_classes)
            b_avg = tf.where(one_hot < 1.0,
                             tf.exp(self._s * logits),
                             tf.zeros_like(logits))
            b_avg = tf.reduce_mean(tf.reduce_sum(b_avg, axis=1))
            theta_class = tf.gather_nd(
                    theta,
                    tf.stack([
                        tf.range(tf.shape(label)[0]),
                        tf.cast(label, tf.int32)
                    ], axis=1))
            mid_index = tf.shape(theta_class)[0] // 2 + 1
            theta_med = tf.nn.top_k(theta_class, mid_index).values[-1]

            # Since _s is not trainable, this assignment is safe. Also,
            # tf.function ensures that this will run in the right order.
            self._s.assign(
                    tf.math.log(b_avg) /
                    tf.math.cos(tf.minimum(math.pi/4, theta_med)))

            # Return scaled logits
            return self._s * logits